# Claude Tools - Auto Processor\nThis notebook automatically starts processing Claude Tools requests

In [ ]:
#@title Setup and Mount Drive { display-mode: "form" }\nfrom google.colab import drive\nimport time\nprint('🔄 Mounting Google Drive...')\ndrive.mount('/content/drive')\nprint('✅ Drive mounted!')\n\n# Install dependencies\n!pip install -q google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client\nprint('✅ Dependencies installed')

In [ ]:
#@title Claude Tools Processor { display-mode: "form" }\nimport os\nimport json\nimport time\nimport traceback\nfrom datetime import datetime\nfrom google.colab import auth\nfrom googleapiclient.discovery import build\nfrom googleapiclient.http import MediaIoBaseUpload\nimport io\n\n# Configuration\nFOLDER_ID = '1ruRdOXUJi16sHgDtpgxqtR7A7g7J8PpA'\n\nclass ColabProcessor:\n    def __init__(self, folder_id):\n        self.folder_id = folder_id\n        self.drive_service = None\n        self.processed_requests = set()\n        \n    def initialize(self):\n        auth.authenticate_user()\n        self.drive_service = build('drive', 'v3')\n        print(f'✅ Initialized with folder: {self.folder_id}')\n        \n    def execute_code(self, code):\n        from io import StringIO\n        import sys\n        old_stdout = sys.stdout\n        sys.stdout = StringIO()\n        try:\n            exec(code, {'__name__': '__main__'})\n            output = sys.stdout.getvalue()\n            return {'status': 'success', 'output': output, 'timestamp': time.time()}\n        except Exception as e:\n            return {'status': 'error', 'error': str(e), 'traceback': traceback.format_exc(), 'timestamp': time.time()}\n        finally:\n            sys.stdout = old_stdout\n    \n    def list_requests(self):\n        query = f\"'{self.folder_id}' in parents and name contains 'request_' and trashed=false\"\n        results = self.drive_service.files().list(q=query, fields='files(id, name)').execute()\n        return [f for f in results.get('files', []) if f['name'].endswith('.json') and f['id'] not in self.processed_requests]\n    \n    def read_request(self, file_id):\n        content = self.drive_service.files().get_media(fileId=file_id).execute()\n        return json.loads(content.decode('utf-8'))\n    \n    def write_response(self, request_id, response_data):\n        response_name = f'response_{request_id}.json'\n        file_metadata = {'name': response_name, 'parents': [self.folder_id]}\n        media = MediaIoBaseUpload(io.BytesIO(json.dumps(response_data, indent=2).encode('utf-8')), mimetype='application/json')\n        self.drive_service.files().create(body=file_metadata, media_body=media).execute()\n        print(f'✅ Response written: {response_name}')\n    \n    def process_request(self, request_file):\n        request_id = request_file['name'].replace('request_', '').replace('.json', '')\n        print(f'📋 Processing: {request_id}')\n        try:\n            request_data = self.read_request(request_file['id'])\n            if request_data['type'] == 'execute_code':\n                result = self.execute_code(request_data['code'])\n            else:\n                result = {'status': 'error', 'error': f\"Unknown request type: {request_data['type']}\"}\n            self.write_response(request_id, result)\n            self.processed_requests.add(request_file['id'])\n        except Exception as e:\n            print(f'❌ Error: {e}')\n            self.write_response(request_id, {'status': 'error', 'error': str(e), 'timestamp': time.time()})\n    \n    def run(self, duration=3600, poll_interval=3):\n        print(f'🚀 Starting Auto Processor')\n        print(f'⏱️  Running for {duration/60:.0f} minutes')\n        print(f'📁 Monitoring: {self.folder_id}')\n        start_time = time.time()\n        while time.time() - start_time < duration:\n            requests = self.list_requests()\n            if requests:\n                print(f'\\n📨 Found {len(requests)} request(s)')\n                for req in requests:\n                    self.process_request(req)\n            else:\n                print(f'⏳ Waiting... ({datetime.now().strftime(\"%H:%M:%S\")})', end='\\r')\n            time.sleep(poll_interval)\n        print('\\n⏱️  Auto-run completed')\n\n# Auto-start processor\nprint('🤖 Claude Tools Auto Processor')\nprint('=' * 40)\nprocessor = ColabProcessor(FOLDER_ID)\nprocessor.initialize()\nprocessor.run(duration=3600, poll_interval=2)  # Run for 1 hour